# New Script to Combine and Clean the Data Into a Single CSV for the Regression(s)

## Packages

In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go

import statsmodels.api as sm

from stargazer.stargazer import Stargazer

## CSV Files

In [2]:
health = pd.read_csv('oecd health.csv')

alc = pd.read_csv('alcohol.csv', usecols=['TIME', 'Value'])
alc.TIME = alc.TIME.astype(int)
alc.columns = ['TIME', 'alc']

elderly = pd.read_csv('elderly.csv', usecols=['TIME', 'Value'])
elderly.TIME = elderly.TIME.astype(int)
elderly.columns = ['TIME', 'elderly']

income = pd.read_csv('HHDI.csv', usecols=['TIME', 'Value'])
income.TIME = income.TIME.astype(int)
income.columns = ['TIME', 'income']

life_exp = pd.read_csv('life expectancy.csv', usecols=['TIME', 'Value'])
life_exp.TIME = life_exp.TIME.astype(int)
life_exp.columns = ['TIME', 'life_exp']

life_exp65m = pd.read_csv('life_exp at 65 men.csv', usecols=['TIME', 'Value'])
life_exp65m.TIME = life_exp65m.TIME.astype(int)
life_exp65m.columns = ['TIME', 'life_exp65']

life_exp65w = pd.read_csv('life_exp at 65 women.csv', usecols=['TIME', 'Value'])
life_exp65w.TIME = life_exp65w.TIME.astype(int)
life_exp65w.columns = ['TIME', 'life_exp65']

unemp = pd.read_csv('unemployment.csv', usecols=['TIME', 'Value'])
unemp.TIME = unemp.TIME.astype(int)
unemp.columns = ['TIME', 'unemp']

In [3]:
# averaging out the life expenctancy at 65 between men and women, then creating that dataset
life_exp65 = []
for time in life_exp65m.index.tolist():
    avg = np.mean([life_exp65m.loc[time,'life_exp65'], life_exp65w.loc[time,'life_exp65']])
    life_exp65.append(avg)
    
life_exp65 = pd.DataFrame([life_exp65m.TIME.tolist(), life_exp65]).transpose()
life_exp65.columns = ['TIME', 'life_exp65']
life_exp65.TIME = life_exp65.TIME.astype(int)

# Cleaning and Combining Data Together

In [4]:
# merging all the datasets together
full = alc.merge(elderly, on='TIME')
full = full.merge(income, on='TIME')
full = full.merge(life_exp, on='TIME')
full = full.merge(life_exp65, on='TIME')
full = full.merge(unemp, on='TIME')

#renaming the columns correctly
# full.columns = col_names

# dropping null values for missing data
full = full.dropna().reset_index(drop=True)

# adding linear time trend for later
full['t'] = full.index+1

# defining the col_names variable for later
col_names = pd.Series(full.columns).drop(0).tolist()

In [5]:
## adding the dependent variables to the mix

# extracting the relevant data types from the health dataset and forcing them into a new dataset, then adding them to the 
# 'full' dataset
# also creating a list of all the possible responses to keep for later to change the regressions if wanted
responses = []
for sub in health.SUBJECT.unique():
    temp = health[health.SUBJECT==sub]
    for m in temp.MEASURE.unique():
        if m=='PC_HEALTHXP':
            continue
        cols = ['TIME', m+'_'+sub]
        df = temp[temp.MEASURE==m][['TIME', 'Value']]
        df.columns = cols
        full = full.merge(df, on='TIME')
        responses.append(m+'_'+sub)
        
responses = np.array(responses)
responses.sort()

In [6]:
# # creating a list of all the dependent variables of interest for the models
# dependents = []
# for name in full.columns:
#     if name.startswith('PC_GDP'):
#         dependents.append(name)

## Adding in Dummy Variables

### Dummy variable for when the ACA took effect (namely in 2014, but only in 2014 as that created a massive shift)

In [7]:
aca = []
for year in full.TIME:
    if year == 2014:
        aca.append(1)
    else:
        aca.append(0)

full['ACA'] = aca

### Adding in the recession dummy

In [8]:
recession = []
for year in full.TIME:
    if ( 
        (int(year)==1960) or
        (int(year)==1961) or
        (int(year)==1969) or
        (int(year)==1970) or
        (int(year)==1973) or
        (int(year)==1974) or
        (int(year)==1975) or
        (int(year)==1980) or
        (int(year)==1981) or
        (int(year)==1982) or
        (int(year)==1990) or
        (int(year)==1991) or
        (int(year)==2001) or
        (int(year)==2007) or
        (int(year)==2008) or
        (int(year)==2009)
       ):
        recession.append(1)
    else:
        recession.append(0)
        
full['recession'] = recession

### Adding in dummies for health crises, specifically pandemics and epidemics in the US

In [9]:
## adding a variable for the second measles outbreak

measles = []
for year in full.TIME:
    if (int(year)>=1981 and int(year)<=1991):
        measles.append(1)
    else:
        measles.append(0)

full['measles'] = measles

In [10]:
## adding a variable for contaminated water in Milwaukee

water = []
for year in full.TIME:
    if (int(year)==1993):
        water.append(1)
    else:
        water.append(0)

full['water'] = water

In [11]:
## adding dummy for H1N1 in 2009

h1n1 = []
for year in full.TIME:
    if (int(year) == 2009):
        h1n1.append(1)
    else:
        h1n1.append(0)
        
full['H1N1'] = h1n1

In [12]:
## adding a dummy for whooping cough in 2010 and 2014

whoop = []
for year in full.TIME:
    if (int(year)==2010 or int(year)==2014):
        whoop.append(1)
    else:
        whoop.append(0)
        
full['whoop'] = whoop

In [13]:
## adding a dummy for the the HIV/AIDS epidemic

aids = []
for year in full.TIME:
    if (int(year) >= 1981):
        aids.append(1)
    else:
        aids.append(0)

full['AIDS'] = aids

In [14]:
# ensuring the entire dataset has types float
full = full.astype(float)

# Creating the Lists of all the Control Variables, the Variables of Interest, and the Dependent Variables to clean the final dataset

In [15]:
controls = col_names + ['ACA']

interest = ['recession', 'measles', 'water', 'H1N1', 'whoop', 'AIDS']

independent = controls + interest

# the dependent list was already created

In [16]:
# adding in first and second differences to the data so that they can be used in the regression more easily

diffs = []

for name in responses:
    full[name+'_diff'] = full[name].diff()
    full[name+'_diff2'] = full[name].diff().diff()
    diffs.append(name+'_diff')
    diffs.append(name+'_diff2')

In [17]:
# creating the final columns for the dataset that will eventually be uses
cols = controls + interest + responses.tolist() + diffs

In [18]:
# keeping only the relevant columns for the regression and exporting data to a csv

full = full[['TIME']+cols].dropna().reset_index(drop=True)

# adding linear time trend
full['t'] = full.index+1

# Regressions

In [19]:
diff = '_diff'
diff2 = '_diff2'
models_list = []

for var in responses[:4]:
    model = sm.OLS(full.loc[:len(full), var+diff], sm.add_constant(full.loc[:len(full), independent])).fit(cov_type='HC1')
    print(model.summary())
    models_list.append(model)

                              OLS Regression Results                              
Dep. Variable:     PC_GDP_COMPULSORY_diff   R-squared:                       0.985
Model:                                OLS   Adj. R-squared:                  0.978
Method:                     Least Squares   F-statistic:                     3391.
Date:                    Sun, 13 Dec 2020   Prob (F-statistic):           1.59e-44
Time:                            19:08:21   Log-Likelihood:                 38.709
No. Observations:                      46   AIC:                            -47.42
Df Residuals:                          31   BIC:                            -19.99
Df Model:                              14                                         
Covariance Type:                      HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       

C:\Users\leeor\anaconda3\lib\site-packages\statsmodels\base\model.py:1830: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 14, but rank is 11
  warnings.warn('covariance of constraints does not have full '
C:\Users\leeor\anaconda3\lib\site-packages\statsmodels\base\model.py:1830: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 14, but rank is 11
  warnings.warn('covariance of constraints does not have full '
C:\Users\leeor\anaconda3\lib\site-packages\statsmodels\base\model.py:1830: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 14, but rank is 11
  warnings.warn('covariance of constraints does not have full '
C:\Users\leeor\anaconda3\lib\site-packages\statsmodels\base\model.py:1830: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 14, but rank is 11
  warnings.warn('covariance of constraints does not have

In [20]:
# testing first differences for stationarity

# importing the Augmented Dickey-Fuller test from statsmodels package
from statsmodels.tsa.stattools import adfuller

for var in responses[:4]:
    print(var+diff)
    print(adfuller(x=full.loc[0:len(full)-5, var+diff].dropna())[0:2])
    print('')
    

'''PC_GDP first difference data is stationary up until ACA but that is a single blip that is a change in regression.'''

PC_GDP_COMPULSORY_diff
(-4.2872917141839375, 0.00046726826770870895)

PC_GDP_OOPEXP_diff
(-4.441354519713144, 0.00025061332988043336)

PC_GDP_TOT_diff
(-4.629583441248171, 0.00011388772588455262)

PC_GDP_VOLUNTARY_diff
(-4.246061526698728, 0.0005500888510791006)



'PC_GDP first difference data is stationary up until ACA but that is a single blip that is a change in regression.'

# Saving Full Dataset to CSV

In [21]:
# outputting the data to a csv
full.to_csv('full data.csv', index=False)

# Data Summary Statistics

In [22]:
# creating lists with the number of observations, the mean, the standard deviation, and the 5-number summary of each
# independent variable and the variables of interest in the regression, then putting it all into a pandas DataFrame
# that can be exported into a csv file and used in the paper more easily

summary_stats = pd.DataFrame()

for var in independent+[var+diff for var in responses[:4]]:
    temp = []
    temp = temp + [len(full)]
    temp = temp + [full[var].mean()]
    temp = temp + [full[var].std()]
    temp = temp + np.quantile(full[var], [0, .25, .5, .75, 1]).tolist()
    summary_stats[var] = temp

summary_stats = summary_stats.transpose()
summary_stats.columns = ['n', 'mean', 'standard deviation', 'min', '25 percentile', 'median', '75 percentile', 'max']
# summary_stats = summary_stats.transpose()

In [23]:
summary_stats.to_csv('summary stats.csv')

# Regression Summaries and Plots

In [24]:
models_column_names = [name.replace('_', ' ') for name in responses[:4]]

models_results = Stargazer(models_list)
models_results.custom_columns(models_column_names, [1,1,1,1])
models_results.show_model_numbers(False)
models_results.title('Table 2: Results of equation (1) regression on each response variable')
models_results.covariate_order(independent)

models_results

In [25]:
with open('results html.html', 'w') as f:
    f.write(models_results.render_html())

In [26]:
independent2 = col_names + interest

for var in responses[[0, 3]]:
    name = var+diff
    print(name)
    model1 = sm.OLS(full.loc[:len(full), name], sm.add_constant(full.loc[:len(full), independent])).fit(cov_type='HC1')
    model2 = sm.OLS(full.loc[:len(full), name], sm.add_constant(full.loc[:len(full), independent2])).fit(cov_type='HC1')
    print(model1.rsquared - model2.rsquared)
    print('')

PC_GDP_COMPULSORY_diff
0.4113690366718118

PC_GDP_VOLUNTARY_diff
0.3740171146023046

